In [1]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from pretty_html_table import build_table

import psycopg2
from configparser import ConfigParser

import numpy as np
import pandas as pd
import math
from sqlalchemy import create_engine
from psycopg2 import Error

In [2]:
try:
    # connecting to DB
    connection=psycopg2.connect(user="shithi", password="tallykhata03012021_1234", host="192.168.168.52", port="5432", database="tallykhata")
    cursor=connection.cursor()
    
    # extracting data

    postgres_query='''drop table if exists data_vajapora.sms_help;'''
    cursor.execute(postgres_query)
    connection.commit()
    
    postgres_query='''
    create table data_vajapora.sms_help as 
    select
        to_char(to_char(current_date, 'YYYY-MM-01')::date-15, 'Mon-YYYY') as year_month,
        count(case when s.telco_identifier_id in(61, 62) and lower(s.message_body) like '%verification code%' then s.id end) as GP_otp_sms_cnt,
        count(case when s.telco_identifier_id in(61, 62) and lower(s.message_body) not like '%verification code%' then s.id end) as GP_txn_sms_cnt,
        count(case when s.telco_identifier_id in(49) and lower(s.message_body) like '%verification code%' then s.id end) as Mobireach_otp_sms_cnt,
        count(case when s.telco_identifier_id in(49) and lower(s.message_body) not like '%verification code%' then s.id end) as Mobireach_txn_sms_cnt,
        count(case when s.telco_identifier_id in(64, 66) and lower(s.message_body) like '%verification code%' then s.id end) as Banglalink_otp_sms_cnt,
        count(case when s.telco_identifier_id in(64, 66) and lower(s.message_body) not like '%verification code%' then s.id end) as Banglalink_txn_sms_cnt
    from public.t_scsms_message_archive_v2 as s
    where
        upper(s.channel) in('TALLYKHATA_OTP','TALLYKHATA_TXN') and upper(trim(s.bank_name)) = 'SURECASH'
        and s.telco_identifier_id in(66, 64,61,62,49) and upper(s.message_status) in ('SUCCESS','0')
        and to_char(s.request_time::date, 'YYYY-MM')=to_char(to_char(current_date, 'YYYY-MM-01')::date-15, 'YYYY-MM');
    '''
    cursor.execute(postgres_query)
    connection.commit()
    
    print("Data generated for 1 month.")
    
    postgres_query='''
    -- summarized data 1
    select 'activity month' "Details", year_month::text "Value" from data_vajapora.sms_help
    -- total SMS
    union select 'total OTP SMS count' "Details", trim(to_char(GP_otp_sms_cnt+Mobireach_otp_sms_cnt+Banglalink_otp_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'total txn SMS count' "Details", trim(to_char(GP_txn_sms_cnt+Mobireach_txn_sms_cnt+Banglalink_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    order by 1 asc; 
    '''
    cursor.execute(postgres_query)
    colnames=[desc[0] for desc in cursor.description]
    df_fetched=pd.DataFrame(cursor.fetchall(), columns=colnames)
    df_fetched1=build_table(df_fetched, 'blue_light')
    
    postgres_query='''
    -- summarized data 2
    select 'activity month' "Details", year_month::text "Value" from data_vajapora.sms_help
    -- GP
    union select 'GP total SMS count' "Details", trim(to_char(GP_otp_sms_cnt+GP_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'GP OTP SMS count' "Details", trim(to_char(GP_otp_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'GP txn SMS count' "Details", trim(to_char(GP_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    -- Mobireach
    union select 'Mobireach total SMS count' "Details", trim(to_char(Mobireach_otp_sms_cnt+Mobireach_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'Mobireach OTP SMS count' "Details", trim(to_char(Mobireach_otp_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'Mobireach txn SMS count' "Details", trim(to_char(Mobireach_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    -- Banglalink
    union select 'Banglalink total SMS count' "Details", trim(to_char(Banglalink_otp_sms_cnt+Banglalink_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'Banglalink OTP SMS count' "Details", trim(to_char(Banglalink_otp_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    union select 'Banglalink txn SMS count' "Details", trim(to_char(Banglalink_txn_sms_cnt, '999G999G999G999G999G999G999')) "Value" from data_vajapora.sms_help
    order by 1 asc;  
    '''
    cursor.execute(postgres_query)
    colnames=[desc[0] for desc in cursor.description]
    df_fetched=pd.DataFrame(cursor.fetchall(), columns=colnames)
    df_fetched2=build_table(df_fetched, 'blue_light')
    
    print("Data summarized for 1 month.")
    
# handling exception
except (Exception, psycopg2.Error) as error:
    print("Error!", error)
    
# closing connection to DB
finally:
    if(connection):
        cursor.close()
        connection.close()
        print("\nPostgreSQL connection is safely closed.")

Data generated for 1 month.
Data summarized for 1 month.

PostgreSQL connection is safely closed.


In [7]:
def send_mail():

    message = MIMEMultipart()

    # subject, from, to 
    message['Subject'] = 'Monthly SMS Report'
    message['From'] = 'shithi.maitra@surecash.net'
    To_receiver = ['hasibsiddique@surecash.net']
    Cc_receiver = ['fahim.wahid@surecash.net', 'data@surecash.net', 'khalid@surecash.net', 'awlad@surecash.net']                        

    message['To'] = ";".join(To_receiver)
    message['Cc'] = ";".join(Cc_receiver)
    receiver = To_receiver + Cc_receiver

    body = f'''
     Hello Bhaiya,<br><br>
     Please find below last month's SMS consumption summary by TallyKhata.<br>
     '''+df_fetched1+'''
     Also find below breakdown of last month's SMS consumptions by different operators.<br>
     '''+df_fetched2+'''
     Thanks,<br>
     Data and BI, TallyKhata
    '''
    message.attach(MIMEText(body, "html"))
    msg_body = message.as_string()

    connection=smtplib.SMTP('smtp.gmail.com', 587)
    connection.ehlo()       # identifying ourselves with the mail server we're using
    connection.starttls()   # encrypting the trafic
    connection.ehlo()       # re-identifying ourselves as an encrypted connection

    # connection.login('shithi.maitra@surecash.net', 'Maitra300795') # logging in
    connection.login('shithi.maitra@surecash.net', 'eysbrqowefwxluyz') # logging in with 'generated app password'
    connection.sendmail(message['From'], receiver, msg_body)
    connection.quit()
    print("Mail sent successfully.")


In [8]:
send_mail()

Mail sent successfully.
